In [5]:
import torch
from torchtext import data
from torchtext import datasets
import random
from utils import epoch_time, count_parameters, categorical_accuracy

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.long)

In [6]:
fields = [('text', TEXT), ('label', LABEL)]

train_data, test_data = data.TabularDataset.splits(
                                        path = 'data',
                                        train = 'trial.tsv',
                                        test = 'test.tsv',
                                        format = 'tsv',
                                        fields = fields,
                                        skip_header = True
)
train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [7]:
vars(train_data[-1])

{'text': ['Rest',
  'from',
  'your',
  'worries',
  'found',
  'here',
  '.',
  '|',
  ':',
  '@user',
  '#',
  'tnstateparks',
  '@',
  'Radnor',
  'Lake',
  'Natural',
  '…'],
 'label': '10'}

In [8]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

In [9]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    sort_key=lambda x: len(x.text),
    batch_size = BATCH_SIZE, 
    sort_within_batch = True,
    device = device)

In [10]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(text))
        
        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        #hidden = [batch size, hid dim * num directions]
            
        return self.fc(hidden.squeeze(0))

In [11]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = len(LABEL.vocab)
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

In [12]:
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,820,604 trainable parameters


In [13]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 100])


In [14]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.6610, -0.0730,  0.9238,  ..., -0.2256,  0.8148, -0.4405],
        ...,
        [ 0.1128,  2.0460,  0.8266,  ...,  0.7674, -0.5384, -1.8739],
        [ 1.7783,  1.1309,  0.0594,  ..., -0.2007, -0.7234,  0.1532],
        [ 0.0433,  0.3691, -2.7507,  ...,  1.7293, -1.0242,  0.0297]])

In [15]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.6610, -0.0730,  0.9238,  ..., -0.2256,  0.8148, -0.4405],
        ...,
        [ 0.1128,  2.0460,  0.8266,  ...,  0.7674, -0.5384, -1.8739],
        [ 1.7783,  1.1309,  0.0594,  ..., -0.2007, -0.7234,  0.1532],
        [ 0.0433,  0.3691, -2.7507,  ...,  1.7293, -1.0242,  0.0297]])


## Train model

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

In [19]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = categorical_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [20]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = categorical_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [23]:
N_EPOCHS = 50

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 6s
	Train Loss: 1.983 | Train Acc: 39.46%
	 Val. Loss: 2.361 |  Val. Acc: 32.73%
Epoch: 02 | Epoch Time: 0m 5s
	Train Loss: 1.943 | Train Acc: 40.35%
	 Val. Loss: 2.357 |  Val. Acc: 32.69%
Epoch: 03 | Epoch Time: 0m 6s
	Train Loss: 1.909 | Train Acc: 41.49%
	 Val. Loss: 2.367 |  Val. Acc: 32.55%
Epoch: 04 | Epoch Time: 0m 6s
	Train Loss: 1.873 | Train Acc: 42.30%
	 Val. Loss: 2.385 |  Val. Acc: 32.11%
Epoch: 05 | Epoch Time: 0m 5s
	Train Loss: 1.834 | Train Acc: 43.65%
	 Val. Loss: 2.424 |  Val. Acc: 31.68%
Epoch: 06 | Epoch Time: 0m 5s
	Train Loss: 1.808 | Train Acc: 44.31%
	 Val. Loss: 2.424 |  Val. Acc: 31.69%
Epoch: 07 | Epoch Time: 0m 6s
	Train Loss: 1.765 | Train Acc: 45.33%
	 Val. Loss: 2.457 |  Val. Acc: 31.19%
Epoch: 08 | Epoch Time: 0m 6s
	Train Loss: 1.730 | Train Acc: 46.06%
	 Val. Loss: 2.480 |  Val. Acc: 31.42%
Epoch: 09 | Epoch Time: 0m 6s
	Train Loss: 1.698 | Train Acc: 47.11%
	 Val. Loss: 2.507 |  Val. Acc: 31.75%
Epoch: 10 | Epoch Time: 0m 6